In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
import math
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['metadata_context_role'] == 'StudentEnrollment']
assignmentsDataSubmissionsData.drop(['body_role', 'body_asset_name', 'collected_at', 'metadata_context_role'], axis = 1, inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.rename(columns = {"body_asset_id": "body_assignment_id"})
assignmentsDataSubmissionsData

In [ ]:
assignmentsInfoFilesPath = str(input())

In [ ]:
assignmentsInfoFiles = glob.glob(assignmentsInfoFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsInfoData = pd.DataFrame()
for assignmentsInfoFile in assignmentsInfoFiles:
    assignmentsInfoData = assignmentsInfoData.append(pd.read_csv(assignmentsInfoFile), ignore_index = True)
    
assignmentsInfoData

In [ ]:
assignmentsInfoData['metadata_event_time'] = assignmentsInfoData['metadata_event_time'].apply(isoparse)
assignmentsInfoData = assignmentsInfoData.sort_values(by='metadata_event_time').drop_duplicates(subset=['metadata_context_id', 'body_assignment_id', "body_context_id"], keep="last").reset_index()
assignmentsInfoData

In [ ]:
assignmentsInfoData.groupby(['body_submission_types'])['metadata_event_time'].count().to_frame('total').reset_index()

In [ ]:
assignmentsInfoData = assignmentsInfoData.loc[~(assignmentsInfoData['body_submission_types'].isin(['none', 'discussion_topic', 'not_graded', 'online_quiz']))]
assignmentsInfoData

In [ ]:
#assignmentsInfoData['body_due_at'] = assignmentsInfoData['body_due_at'].apply(parseTimestamp)
#assignmentsInfoData['body_lock_at'] = assignmentsInfoData['body_lock_at'].apply(parseTimestamp)
#assignmentsInfoData
assignmentsInfoData.drop(['body_due_at', 'body_lock_at', 'index', 'metadata_event_time', 'body_context_id'], axis = 1, inplace = True)
assignmentsInfoData

In [ ]:
'''
def fixDueDate(df):
    if(pd.isnull(df['body_due_at'])):
        if(pd.isnull(df['body_lock_at'])):
            return endDate
        else:
            return df['body_lock_at']
    else:
        return df['body_due_at']
    
assignmentsInfoData['body_due_at'] = assignmentsInfoData.apply(fixDueDate, axis = 1)
assignmentsInfoData.drop(['body_lock_at', 'index'], axis = 1, inplace = True)
assignmentsInfoData
'''

In [ ]:
#assignmentsInfoData = assignmentsInfoData.loc[(assignmentsInfoData['body_due_at'] >= startDate) & (assignmentsInfoData['body_due_at'] <= endDate)]
#assignmentsInfoData

In [ ]:
assignmentsInfoData.to_csv('CourseAssignmentDueDates_Intercession_2022.csv', header = True, index = False)

In [ ]:
#assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(assignmentsInfoData[['metadata_context_id', 'body_assignment_id', 'body_due_at']], on=['metadata_context_id', 'body_assignment_id'])
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(assignmentsInfoData, on=['metadata_context_id', 'body_assignment_id'])
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(str)
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.sort_values(by=['metadata_context_id', 'metadata_user_id', 'metadata_event_time'])
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData.groupby(['metadata_context_id', 'metadata_user_id']).agg({"metadata_event_time": list}).reset_index().to_csv("AssignmentAccesses_Intercession_2022.csv", header = True, index = False)